## Deployment Video Link
https://youtu.be/XyJ-gpqzbkw

In [ ]:
!pip uninstall tensorflow
!pip install tensorflow==2.4.2

Found existing installation: tensorflow 2.7.0
Uninstalling tensorflow-2.7.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-2.7.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.7.0
     |████████████████████████████████| 394.5 MB 30 kB/s 
     |████████████████████████████████| 462 kB 62.5 MB/s 
     |████████████████████████████████| 3.8 MB 43.8 MB/s 
     |████████████████████████████████| 132 kB 60.6 MB/s 
     |████████████████████████████████| 2.9 MB 39.9 MB/s 
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=68725 sha256=4bce7b7b40f0fa8700f7751b1afed4b0ddb26be102a6683ee667

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# importing all the Libraries
import pickle
import tensorflow as tf
from tensorflow.keras.models import load_model
import re
from numpy import zeros, argmax
from tensorflow.keras.preprocessing.sequence import pad_sequences
    

In [ ]:
# Loading the required models for prediction
test_decoder_model = load_model('/content/drive/MyDrive/Test_Decoder_Model.h5')
test_encoder_model = load_model('/content/drive/MyDrive/Test_Encoder_Model.h5')
    
file = open('/content/drive/MyDrive/GmailSmartCompose_Tokenizer.pickle','rb')
Tokenizer = pickle.load(file)

In [ ]:
# initializing the required variables
max_length_in = 21    
max_length_out = 19

index_to_word = dict(map(reversed, Tokenizer.word_index.items()))

In [ ]:
def tokenize_text(text):
  """returns the text after padding"""
  text = '<start> ' + text.lower() + ' <end>'
  text_tensor = Tokenizer.texts_to_sequences([text])
  text_tensor = pad_sequences(text_tensor, maxlen=max_length_in, padding="post")
  return text_tensor

def decode_sequence_attention(input_tensor):
  """returns the predicted text for given input text"""
    # Encode the input as state vectors.
    enc_out, state = test_encoder_model.predict(input_tensor)

    target_seq = zeros((1, 1))
    target_seq[0, 0] =  Tokenizer.word_index['<start>']
    curr_word = "<start>"
    decoded_sentence = ''
    i = 0
    while curr_word != "<end>" and i < (max_length_out - 1):
        
        output_tokens, h = test_decoder_model.predict([target_seq, state, enc_out])

        curr_token = argmax(output_tokens[0, 0])

        if (curr_token == 0):
          break;

        curr_word = index_to_word[curr_token]

        decoded_sentence += ' ' + curr_word
        target_seq[0, 0] = curr_token
        state = h
        i += 1

    return decoded_sentence[:-6]

In [ ]:
# predicting the output 
input_text = input('Enter the text : ')
predicted_text = decode_sequence_attention(tokenize_text(input_text))
predicted_text = input_text + ' ' + predicted_text
predicted_text = re.sub(r'[" "]+', " ", predicted_text)
print('The predicted sentence is :', '\' ' + predicted_text + ' \'')

Enter the text : call me if
The predicted sentence is : ' call me if you have any questions '
